### Integração com APIs: Feriados e Tempo

#### Imports

In [30]:
import requests
import pandas as pd

#### Endpoint - Status da Resposta

##### Public Holiday API

In [31]:
url = "https://date.nager.at/api/v3/PublicHolidays/2024/BR"

response = requests.get(url)

# Verificar se a resposta foi bem-sucedida
if response.status_code == 200:
    holidays = response.json()
    print(holidays)  # Exibe a lista de feriados
else:
    print(f"Erro na solicitação: {response.status_code}")
    print(f"Conteúdo da resposta: {response.text}")

[{'date': '2024-01-01', 'localName': 'Confraternização Universal', 'name': "New Year's Day", 'countryCode': 'BR', 'fixed': False, 'global': True, 'counties': None, 'launchYear': None, 'types': ['Public']}, {'date': '2024-02-12', 'localName': 'Carnaval', 'name': 'Carnival', 'countryCode': 'BR', 'fixed': False, 'global': True, 'counties': None, 'launchYear': None, 'types': ['Bank', 'Optional']}, {'date': '2024-02-13', 'localName': 'Carnaval', 'name': 'Carnival', 'countryCode': 'BR', 'fixed': False, 'global': True, 'counties': None, 'launchYear': None, 'types': ['Bank', 'Optional']}, {'date': '2024-03-29', 'localName': 'Sexta-feira Santa', 'name': 'Good Friday', 'countryCode': 'BR', 'fixed': False, 'global': True, 'counties': None, 'launchYear': None, 'types': ['Public']}, {'date': '2024-03-31', 'localName': 'Domingo de Páscoa', 'name': 'Easter Sunday', 'countryCode': 'BR', 'fixed': False, 'global': True, 'counties': None, 'launchYear': None, 'types': ['Public']}, {'date': '2024-04-21', '

##### Open-Meteo Historical Weather API

In [32]:
# URL da API Open-Meteo
url_weather = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": -22.901449,  # Latitude do Rio de Janeiro
    "longitude": -43.178921,  # Longitude do Rio de Janeiro
    "start_date": "2024-01-01",
    "end_date": "2024-08-01",
    "daily": "temperature_2m_max,temperature_2m_min,weathercode",
    "timezone": "America/Sao_Paulo"
}

# Fazer a requisição à API
response_weather = requests.get(url_weather, params=params)

# Verificar se a requisição foi bem-sucedida e obter os dados
weather_data = response_weather.json()


In [33]:
# Verificar as chaves disponíveis nos dados retornados
print(weather_data.keys())

dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'daily_units', 'daily'])


In [34]:
# Visualizar os dados diários para entender a estrutura
print(weather_data['daily'])

{'time': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-20', '2024-01-21', '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25', '2024-01-26', '2024-01-27', '2024-01-28', '2024-01-29', '2024-01-30', '2024-01-31', '2024-02-01', '2024-02-02', '2024-02-03', '2024-02-04', '2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16', '2024-02-17', '2024-02-18', '2024-02-19', '2024-02-20', '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-24', '2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28', '2024-02-29', '2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05', '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-09', '2024-03-10', '2024-03-1

#### 1. Quantos feriados há no Brasil em todo o ano de 2024?

In [35]:
url = "https://date.nager.at/api/v3/PublicHolidays/2024/BR"
response = requests.get(url)

if response.status_code == 200:
    holidays = response.json()
    total_holidays = len(holidays)
    print(f"Número total de feriados em 2024: {total_holidays}")
else:
    print(f"Erro na solicitação: {response.status_code}")

Número total de feriados em 2024: 15


#### 2. Qual mês de 2024 tem o maior número de feriados?

In [36]:
# Converter para um DataFrame
df_holidays = pd.DataFrame(holidays)

# Extrair o mês dos feriados
df_holidays['month'] = pd.to_datetime(df_holidays['date']).dt.month

# Contar o número de feriados por mês
holidays_per_month = df_holidays['month'].value_counts().sort_index()
most_holidays_month = holidays_per_month.idxmax()

print(f"O mês com o maior número de feriados é {most_holidays_month}, com {holidays_per_month[most_holidays_month]} feriados.")

O mês com o maior número de feriados é 11, com 3 feriados.


#### 3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?

In [37]:
# Filtrar apenas os feriados que caem de segunda a sexta-feira
df_holidays['weekday'] = pd.to_datetime(df_holidays['date']).dt.weekday
weekdays_holidays = df_holidays[df_holidays['weekday'] < 5]

weekday_holidays_count = len(weekdays_holidays)
print(f"Número de feriados que caem em dias de semana: {weekday_holidays_count}")


Número de feriados que caem em dias de semana: 10


#### 4. Qual foi a temperatura média em cada mês?

In [38]:
# Converter a resposta para um DataFrame
df_weather = pd.DataFrame(weather_data['daily'])
df_weather['date'] = pd.to_datetime(df_weather['time'])
df_weather['month'] = df_weather['date'].dt.month

# Calcular a temperatura média para cada mês
df_weather['avg_temp'] = (df_weather['temperature_2m_max'] + df_weather['temperature_2m_min']) / 2
monthly_avg_temp = df_weather.groupby('month')['avg_temp'].mean()

print("Temperatura média em cada mês:")
print(monthly_avg_temp)

Temperatura média em cada mês:
month
1    27.104839
2    27.722414
3    26.891935
4    25.650000
5    25.583871
6    23.236667
7    21.674194
8    21.800000
Name: avg_temp, dtype: float64


#### 5. Qual foi o tempo predominante em cada mês nesse período?

In [39]:
# Converter a resposta para um DataFrame
df_weather_codes = pd.DataFrame(weather_data['daily'])
df_weather_codes['date'] = pd.to_datetime(df_weather_codes['time'])
df_weather_codes['month'] = df_weather_codes['date'].dt.month

# Determinar o tempo predominante por mês
weather_code_monthly = df_weather_codes.groupby('month')['weathercode'].agg(lambda x: x.value_counts().idxmax())

print("Tempo predominante em cada mês:")
print(weather_code_monthly)


Tempo predominante em cada mês:
month
1    63
2    51
3    51
4     3
5    51
6     1
7     3
8     2
Name: weathercode, dtype: int64


1: Céu claro (ensolarado)

2: Parcialmente nublado

3: Nublado (completamente nublado)

51: Chuvisco

63: Chuva moderada


#### 6. Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

In [41]:
# Coluna 'date' esteja no formato datetime em ambos os DataFrames
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_weather['date'] = pd.to_datetime(df_weather['time'])

# Filtrar feriados dentro do período
holidays_within_period = df_holidays[(df_holidays['date'] >= "2024-01-01") & (df_holidays['date'] <= "2024-08-01")]

# Junte os dados de feriados com os dados de clima
holidays_weather = pd.merge(holidays_within_period, df_weather, left_on='date', right_on='date', how='left')

print(holidays_weather[['date', 'name', 'avg_temp', 'weathercode']])


        date                                  name  avg_temp  weathercode
0 2024-01-01                        New Year's Day     24.85           51
1 2024-02-12                              Carnival     30.80            1
2 2024-02-13                              Carnival     31.15           51
3 2024-03-29                           Good Friday     25.70           61
4 2024-03-31                         Easter Sunday     24.70           53
5 2024-04-21                            Tiradentes     23.50            2
6 2024-05-01                            Labour Day     28.35            0
7 2024-05-30                        Corpus Christi     20.90           51
8 2024-07-09  Constitutionalist Revolution of 1932     21.95           53


#### 7. Considere as seguintes suposições:

- O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
- Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
- O cidadão carioca só vai à praia quando não está com frio;
- O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).


Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

In [45]:
# Definindo as condições para um feriado "não aproveitável"
holidays_weather['non_approvable'] = (
    (holidays_weather['avg_temp'] < 20) |  
    (holidays_weather['weathercode'].isin([3, 51, 53, 55, 61, 63, 65, 80, 81, 82, 95, 96, 99]))  
)

# Filtrando os feriados "não aproveitáveis"
non_approvable_holidays = holidays_weather[holidays_weather['non_approvable']]

# Verificando se houve algum feriado "não aproveitável"
if len(non_approvable_holidays) > 0:
    print("Feriados não aproveitáveis:")
    print(non_approvable_holidays[['date', 'name', 'avg_temp', 'weathercode']])
else:
    print("Nenhum feriado não aproveitável encontrado.")


Feriados não aproveitáveis:
        date                                  name  avg_temp  weathercode
0 2024-01-01                        New Year's Day     24.85           51
2 2024-02-13                              Carnival     31.15           51
3 2024-03-29                           Good Friday     25.70           61
4 2024-03-31                         Easter Sunday     24.70           53
7 2024-05-30                        Corpus Christi     20.90           51
8 2024-07-09  Constitutionalist Revolution of 1932     21.95           53


0	Céu claro

1, 2, 3	Principalmente claro, parcialmente nublado e nublado

45, 48	Névoa e depósito de gelo

51, 53, 55	Chuvisco: Intensidade leve, moderada e densa

56, 57	Chuvisco congelante: Intensidade leve e densa

61, 63, 65	Chuva: Intensidade leve, moderada e forte

66, 67	Chuva congelante: Intensidade leve e forte

71, 73, 75	Queda de neve: Intensidade leve, moderada e forte

77	Grãos de neve

80, 81, 82	Pancadas de chuva: Intensidade leve, moderada e violenta

85, 86	Pancadas de neve: Intensidade leve e forte

95	Trovoada: Intensidade leve ou moderada

96, 99	Trovoada com granizo leve e forte

#### 8. Qual foi o feriado "mais aproveitável" de 2024

Considere o melhor par tempo e temperatura.

In [47]:
# Definir uma pontuação de "aproveitabilidade" para cada feriado
# Condições favoráveis: alta temperatura e códigos de tempo favoráveis (1 = Céu claro, 2 = Parcialmente nublado)
holidays_weather['approvable_score'] = holidays_weather['avg_temp'] * holidays_weather['weathercode'].apply(lambda x: 1 if x in [1, 2] else 0)

# Encontrar o feriado com a maior pontuação de "aproveitabilidade"
best_holiday = holidays_weather.loc[holidays_weather['approvable_score'].idxmax()]

# Exibir o feriado mais aproveitável
print(f"O feriado mais aproveitável foi {best_holiday['name']} em {best_holiday['date']} com temperatura média de {best_holiday['avg_temp']}ºC e código de tempo {best_holiday['weathercode']}.")


O feriado mais aproveitável foi Carnival em 2024-02-12 00:00:00 com temperatura média de 30.8ºC e código de tempo 1.
